In [70]:
from ultralytics import YOLO
import torch
import numpy as np
import cv2
from matplotlib import pyplot as plt

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)
model.to(device)
model.train(data="config.yaml", epochs=100, batch=4, imgsz=400, amp=False, close_mosaic=0) 

Ultralytics YOLOv8.0.171  Python-3.11.3 torch-2.0.1+cu118 CUDA:0 (NVIDIA GeForce GTX 1650 Ti, 4096MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=config.yaml, epochs=100, patience=50, batch=4, imgsz=400, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=False, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optim

In [5]:
model = YOLO("best.pt")

In [6]:
metrics = model.val(conf=0.5, iou=0.1)  # evaluate model performance on the validation set
results = model("C:/Users/da4nik/CV Task/test.jpg")

Ultralytics YOLOv8.0.171  Python-3.11.3 torch-2.0.1+cu118 CUDA:0 (NVIDIA GeForce GTX 1650 Ti, 4096MiB)
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients
val: Scanning C:\Users\da4nik\CV Task\data\labels\val.cache... 4 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4/4
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:09<0
                   all          4         34          1      0.936       0.98      0.788
                     0          4         19          1      0.872      0.966      0.682
                     1          4         15          1          1      0.995      0.893
Speed: 2.0ms preprocess, 49.8ms inference, 0.0ms loss, 2.8ms postprocess per image
Results saved to runs\detect\val13

image 1/1 C:\Users\da4nik\CV Task\test.jpg: 256x416 4 0s, 3 1s, 52.0ms
Speed: 4.0ms preprocess, 52.0ms inference, 3.0ms postprocess per image at shape (1, 3, 256, 416)


In [100]:
results = model("C:/Users/da4nik/CV Task/data/images/val/new_tic_15.jpg", conf=0.4, iou=0.1)


image 1/1 C:\Users\da4nik\CV Task\data\images\val\new_tic_15.jpg: 320x416 4 0s, 3 1s, 94.0ms
Speed: 4.0ms preprocess, 94.0ms inference, 15.0ms postprocess per image at shape (1, 3, 320, 416)


In [101]:
for r in results:
    labels = r.boxes.cls.cpu().numpy()
    coordinates = r.boxes.xywh.cpu().numpy()

In [103]:
centr = coordinates[:,0:2]

In [107]:
mins = []
for c in centr:
    lst = []
    for c2 in centr:
        if np.array_equal(c, c2): continue 
        dist = np.linalg.norm(c - c2)
        lst.append(dist)
    mins.append(min(lst))

In [108]:
def Get_true(centr, labels, alpha):
    mins = []
    for c in centr:
        lst = []
        for c2 in centr:
            if np.array_equal(c, c2): continue 
            dist = np.linalg.norm(c - c2)
            lst.append(dist)
        mins.append(min(lst))

    median = sorted(mins)[len(mins)//2]

    filtr_idx = []
    for idx, m in enumerate(mins):
        if m/median > alpha:
            filtr_idx.append(idx)
            
    filtr_centr = np.delete(centr, filtr_idx, axis=0)
    filtr_labels = np.delete(labels, filtr_idx)
    
    return filtr_centr, filtr_labels

In [109]:
c, l = Get_true(centr, labels, 2)

In [111]:
arr_y = np.zeros((9, 9))
arr_x = np.zeros((9, 9))
for idx1, i in enumerate(c):
    for idx2, j in enumerate(c):
        arr_y[idx1, idx2] = np.abs(i[1] - j[1])
        arr_x[idx1, idx2] = np.abs(i[0] - j[0])

In [112]:
bound_box = coordinates[:,2:]

In [114]:
xmax, ymax = np.max(c, axis=0)
xmin, ymin = np.min(c, axis=0)

In [115]:
cell_height = (ymax - ymin) / 2
cell_width = (xmax - xmin) / 2

In [116]:
cell_height

231.34683227539062

In [117]:
cell_width

200.64573669433594

In [123]:
def map_coordinates_tocells(coordinates, labels, min_y, min_x, cell_height, cell_width):
    # Initialize a 3x3 grid to represent the Tic Tac Toe board
    grid = [[' ' for _ in range(3)] for _ in range(3)]
    rc = []
    xy = [[' ' for _ in range(3)] for _ in range(3)]
    # Iterate through the coordinates and labels
    for (x, y), label in zip(coordinates, labels):
        # Calculate the row and column based on the coordinates
        row = round((y - min_y) / cell_height)
        col = round((x - min_x) / cell_width)
        rc.append([row, col])
        # Update the grid with the label (X or O)
        grid[row][col] = label
        xy[row][col] = [x, y]

    return grid, xy 

In [124]:
map_coordinates_tocells(c, l, ymin, xmin, cell_height, cell_width)

([[1.0, ' ', 1.0], [' ', 0.0, 1.0], [0.0, 0.0, 0.0]],
 [[[302.9353, 270.46405], ' ', [642.64734, 302.75348]],
  [' ', [465.3329, 455.77356], [651.3967, 470.75522]],
  [[285.45602, 688.02637], [480.04492, 707.16187], [686.7475, 733.1577]]])

In [ ]:
def find_winner_line(grid):
    # Check rows
    for row in range(3):
        if grid[row][0] == grid[row][1] == grid[row][2] and grid[row][0] != " ":
            return [(row, 0), (row, 1), (row, 2)]

    # Check columns
    for col in range(3):
        if grid[0][col] == grid[1][col] == grid[2][col] and grid[0][col] != " ":
            return [(0, col), (1, col), (2, col)]

    # Check diagonals
    if grid[0][0] == grid[1][1] == grid[2][2] and grid[0][0] != " ":
        return [(0, 0), (1, 1), (2, 2)]
    if grid[0][2] == grid[1][1] == grid[2][0] and grid[0][2] != " ":
        return [(0, 2), (1, 1), (2, 0)]

    return None  # No winner line found

In [127]:
def Get_coords(centr, labels):

    
    def Get_true(centr, labels, alpha):
        mins = []
        for c in centr:
            lst = []
            for c2 in centr:
                if np.array_equal(c, c2): continue 
                dist = np.linalg.norm(c - c2)
                lst.append(dist)
            mins.append(min(lst))
    
        median = sorted(mins)[len(mins)//2]
    
        filtr_idx = []
        for idx, m in enumerate(mins):
            if m/median > alpha:
                filtr_idx.append(idx)
                
        filtr_centr = np.delete(centr, filtr_idx, axis=0)
        filtr_labels = np.delete(labels, filtr_idx)
        
        return filtr_centr, filtr_labels

    
    def map_coordinates_tocells(coordinates, labels, min_y, min_x, cell_height, cell_width):
        # Initialize a 3x3 grid to represent the Tic Tac Toe board
        grid = [[' ' for _ in range(3)] for _ in range(3)]
        xy = [[' ' for _ in range(3)] for _ in range(3)]
        # Iterate through the coordinates and labels
        for (x, y), label in zip(coordinates, labels):
            # Calculate the row and column based on the coordinates
            row = round((y - min_y) / cell_height)
            col = round((x - min_x) / cell_width)
            # Update the grid with the label (X or O)
            grid[row][col] = label
            xy[row][col] = [x, y]
    
        return grid, xy


    def find_winner_line(grid):
        # Check rows
        for row in range(3):
            if grid[row][0] == grid[row][1] == grid[row][2] and grid[row][0] != " ":
                return [(row, 0), (row, 1), (row, 2)]
    
        # Check columns
        for col in range(3):
            if grid[0][col] == grid[1][col] == grid[2][col] and grid[0][col] != " ":
                return [(0, col), (1, col), (2, col)]
    
        # Check diagonals
        if grid[0][0] == grid[1][1] == grid[2][2] and grid[0][0] != " ":
            return [(0, 0), (1, 1), (2, 2)]
        if grid[0][2] == grid[1][1] == grid[2][0] and grid[0][2] != " ":
            return [(0, 2), (1, 1), (2, 0)]
    
        return None  # No winner line found


    c, l = Get_true(centr, labels, 2)
    
    xmax, ymax = np.max(c, axis=0)
    xmin, ymin = np.min(c, axis=0)

    cell_height = (ymax - ymin) / 2
    cell_width = (xmax - xmin) / 2

    grid, xy = map_coordinates_tocells(c, l, ymin, xmin, cell_height, cell_width)

    win = find_winner_line(grid)

    if win == None: return None

    xy1 = xy[win[0][0]][win[0][1]]
    xy2 = xy[win[2][0]][win[2][1]]
    return xy1, xy2

In [ ]:
model.val(conf=0.4, iou=0.1, batch=2)

Ultralytics YOLOv8.0.171  Python-3.11.3 torch-2.0.1+cu118 CUDA:0 (NVIDIA GeForce GTX 1650 Ti, 4096MiB)
val: Scanning C:\Users\da4nik\CV Task\data\labels\val.cache... 4 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4/4
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:09<0
